In [15]:
from sortedcontainers import SortedList
from bisect import bisect_left, bisect_right

def ksmallest(pivot, k):
    i = A.bisect_left(pivot) - 1
    if k >= len(A)-i-1: return -1
    return A[i+k]

def klargest(pivot, k):
    i = A.bisect_left(pivot)
    if k >= i: return -1
    return A[i-k]


if __name__ == '__main__':
    with open('input.txt', 'r') as f:
        # Q = int(input())
        Q = int(f.readline().strip())
        A = SortedList()  
        A.add(0)
        A.add(1000000000000000001)
        for _ in range(Q):
            # query = list(map(int, input().split()))
            query = list(map(int, f.readline().split()))
            if query[0] == 1:
                A.add(query[1])
            elif query[0] == 2:
                print(klargest(query[1],query[2]))
            else:
                print(ksmallest(query[1],query[2]))
              

20
20
30
-1
-1
1


In [43]:
"""
A solution with python array objects and bisect
"""

from bisect import bisect_left, bisect_right, insort_left 
from array import array

def sequence_query(arr):
  with open('input_stress.txt', 'r') as f:
    Q = int(f.readline().strip())
    # arr = []
    # arr = array("i", [])

    queries = [list(map(int,f.readline().split())) for _ in range(Q)]
    x_values = [queries[i][1] for i in range(Q)]
    x_values = set(x_values)
    x_values = sorted(list(x_values))
    to = {}
    fr = {}
    for i in range(len(x_values)):
      to[x_values[i]] = i
      fr[i] = x_values[i]
    for i in range(Q):
      query = queries[i]
      if query[0] == 1:
        x_index = to[query[1]]
        insort_left(arr, x_index) # nlogn
      elif query[0] == 2:
        x_index = to[query[1]]
        j = bisect_right(arr,x_index)
        k = query[2]
        if k > j:
          return -1
          print(-1)
        else:
          return fr[arr[j-k]]
          print(fr[arr[j-k]])
      else:
        x_index = to[query[1]]
        j = bisect_left(arr,x_index)
        k = query[2]
        if k > len(arr) - j:
          return -1
          print(-1)
        else:
          return fr[arr[j+k-1]]
          print(fr[arr[j+k-1]])

In [47]:
%%timeit -r 1
sequence_query(arr=[])

7.13 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [48]:
%%timeit -r 1
sequence_query(arr=array("i",[]))

1.94 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [34]:
# create a dataset out of curiosity to compare array and list in python
from random import randint
data = [randint(1,100000) for _ in range(200000)]
with open('input_stress.txt', 'w') as f:
  f.write(str(len(data))+'\n')
  for d in data:
    f.write(f"1 {d}\n")

In [68]:
# Fenwick tree

class FenwickTree:
  def __init__(self, N):
    self.sums = [0 for _ in range(N+1)]
  
  def update(self, i, delta):

    while i < len(self.sums):
      self.sums[i] += delta
      i += i & (-i)
      
  def query(self, i):
    res = 0
    while i > 0:
      res += self.sums[i]
      i -= i & (-i)
    return res
  
  def __repr__(self):
    return f"array: {self.sums}"
  
with open('inputs/input.txt', 'r') as f:
  Q = int(f.readline().strip())
  arr = array("i", [])

  queries = [list(map(int,f.readline().split())) for _ in range(Q)]
  x_values = [queries[i][1] for i in range(Q)]
  x_values.append(0)
  x_values = set(x_values)
  x_values = sorted(list(x_values))
  to = {}
  fr = {}
  for i, x in enumerate(x_values):
    to[x] = i
    fr[i] = x
  bit = FenwickTree(len(x_values))
  
  for query in queries:
    if query[0] == 1:
      x_index = to[query[1]]
      bit.update(x_index, 1)
    elif query[0] == 2:
      x, k = query[1:]
      x_index = to[x]
      fz = bit.query(x_index)
      if fz < k:
        print(-1)
      else:
        ok = 0
        ng = x_index + 1
        while ng-ok > 1:
          m = (ok+ng)//2
          if fz - bit.query(m-1) < k:
            ng = m
          else:
            ok = m
        print(fr[ok])
    else:
      x, k = query[1:]
      x_index = to[x]
      fz = bit.query(x_index)
      if bit.query(len(x_values)) - fz < k:
        print(-1)
      else:
        ok = len(x_values)
        ng = x_index - 1
        while ok - ng > 1:
          m = (ok+ng)//2
          if bit.query(m) - fz < k:
            ng = m
          else:
            ok = m
        print(fr[ok])
  print(bit)
  
    

20
20
30
-1
-1
1
array: [0, 1, 2, 0, 4, 1, 1, 0]


4096